In [1]:
from huggingface_hub.utils import HfFolder
from huggingface_hub import login
import os

hf_token = os.getenv("HF_TOKEN")
login(token=hf_token) #enter token

/gpfs3/well/papiez/users/hri611/python/foundation-models-radiology/medgemma_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import BitsAndBytesConfig
import torch

model_variant = "4b-it"  # @param ["4b-it", "27b-text-it"]
model_id = f"google/medgemma-{model_variant}"

use_quantization = True  # @param {type: "boolean"}

# @markdown Set `is_thinking` to `True` to turn on thinking mode. **Note:** Thinking is supported for the 27B variant only.
is_thinking = False  # @param {type: "boolean"}

# If running the 27B variant in Google Colab, check if the runtime satisfies
# memory requirements
if "27b" in model_variant and google_colab:
    if not ("A100" in torch.cuda.get_device_name(0) and use_quantization):
        raise ValueError(
            "Runtime has insufficient memory to run the 27B variant. "
            "Please select an A100 GPU and use 4-bit quantization."
        )

model_kwargs = dict(
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

if use_quantization:
    model_kwargs["quantization_config"] = BitsAndBytesConfig(load_in_4bit=True)

/gpfs3/well/papiez/users/hri611/python/foundation-models-radiology/medgemma_env/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [ ]:
import os
from PIL import Image
from IPython.display import Image as IPImage, display, Markdown

# prompt = "Describe this X-ray"  # @param {type: "string"}
prompt = "Does this chest X-ray show pneumothorax? Answer with only one word yes or no."


In [4]:
system_instruction = "You are an expert radiologist."

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": system_instruction}]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": prompt},
            {"type": "image", "image": image}
        ]
    }
]

In [5]:
from transformers import AutoModelForImageTextToText, AutoProcessor

model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    **model_kwargs,
)
processor = AutoProcessor.from_pretrained(model_id)

Fetching 2 files:  50%|█████████████████████▌                     | 1/2 [01:56<01:56, 116.66s/it]Error while downloading from https://cdn-lfs-us-1.hf.co/repos/3d/df/3ddf3ad8b796b6d5ada0850f03115c053532413d0d668c795d595ebcf54cd23e/1bed82e64597da81b1e663c59c4dae77df93edf8af95221054b541225d4f0313?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00002-of-00002.safetensors%3B+filename%3D%22model-00002-of-00002.safetensors%22%3B&Expires=1750929596&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MDkyOTU5Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzNkL2RmLzNkZGYzYWQ4Yjc5NmI2ZDVhZGEwODUwZjAzMTE1YzA1MzUzMjQxM2QwZDY2OGM3OTVkNTk1ZWJjZjU0Y2QyM2UvMWJlZDgyZTY0NTk3ZGE4MWIxZTY2M2M1OWM0ZGFlNzdkZjkzZWRmOGFmOTUyMjEwNTRiNTQxMjI1ZDRmMDMxMz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=ZKy6f7YOZPyAaq9xLGjIYvpH8ECh3UGmFVZr5Z5ZBXCADdaE4Vb-x%7ERJ6PMh6dV2ortPsyHOPqbXj%7Ei1Xuh-6R069tKwU3l9DD7VWbKtHmqu%7EY5Wp-HOo0IJEiPTuhJ

In [10]:
from pathlib import Path
import glob

root_dir = Path('/well/papiez/users/hri611/python/foundation-models-radiology')

jpg_paths = glob.glob(str(root_dir / 'PTX Head to Head Study Data' / '**/*.jpg'), recursive=True)

In [13]:
import tqdm

responses = []

for path in tqdm.tqdm(jpg_paths):
    image = Image.open(path)
    messages[1]['content'][1]['image'] = image

    inputs = processor.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(model.device, dtype=torch.bfloat16)

    input_len = inputs["input_ids"].shape[-1]

    with torch.inference_mode():
        generation = model.generate(**inputs, max_new_tokens=300, do_sample=False)
        generation = generation[0][input_len:]

    response = processor.decode(generation, skip_special_tokens=True)

    responses.append(response)


  0%|                                                                    | 0/413 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
  0%|▏                                                           | 1/413 [00:00<03:48,  1.80it/s]The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
  0%|▎                                                           | 2/413 [00:01<03:50,  1.78it/s]The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
  1%|▍                                                           | 3/413 [00:06<17:32,  2.57s/

NameError: name 'pd' is not defined

In [23]:
import pandas as pd
medgemma_results_df = pd.DataFrame()
medgemma_results_df['image_path'] = jpg_paths
medgemma_results_df['response'] = responses
medgemma_results_df['binary_score'] = medgemma_results_df['response'].apply(
    lambda x: None if isinstance(x, str) and 'cannot' in x.lower()
    else 1 if isinstance(x, str) and 'yes' in x.lower()
    else 0 if isinstance(x, str) and 'no' in x.lower()
    else None
)
medgemma_results_df.to_csv('ptx_medgemma_scores.csv', index=False)